In [1]:
import os
# 한글 지원
os.putenv('NLS_LANG', '.UTF8')

In [2]:
def ipchecker():    
    import urllib 
    import re
    
    pattern = re.compile(b'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}')
    dyndns = urllib.request.urlopen('http://checkip.dyndns.org').read()
    
    wan = re.search(pattern, dyndns).group().decode('utf-8')
    
    return wan

In [3]:
# base db_connect param

local_ip = '200.1.220.251:1521/xe' # 원내에서 접속할 db_ip
outside_ip = '61.35.152.103:1521/xe' # 원밖에서 접속할 db_ip
localhost = 'localhost:1521/xe'

db_id = 'begopa' 
db_pwd = 'begopa1'

academy_ip = '61.35.152.66'
db_ip = local_ip if academy_ip == ipchecker() else outside_ip

# db_ip = localhost # 집에서 본인의 개인 컴퓨터dptj  오라클에 접속할때는

In [4]:
import pickle
# load object ::
# :: file_nm
# :: columns and data_type
with open("./pickle/col2dict.pickle","rb") as fr:
    col2dict = pickle.load(fr)
with open("./pickle/table_nm_set.pickle","rb") as fr:
    table_nm_set = pickle.load(fr)

In [5]:
path = "./sg-data/"
desc_path = path + 'desc/'
col_path = path + 'col/'
origin_path = path + 'origin/'

file_list = os.listdir(origin_path)

In [6]:
# _, _, _, table_col_set_ = extract_table_columns_set(file_list[0]) # _, :: 임시로 쓰고 버리는 값.

In [7]:
# 자치구별 상권 변화지표 db 에서 한글 컬럼으로 가져오기
# oracle_to_df('자치구별 상권변화지표')
def load_to_df(kr_tbl_nm):
    for key, value in table_nm_set.items():
        if kr_tbl_nm == value:
            table_nm = key
            columns = [ arr[0] for arr in col2dict[key] ]
           
    import cx_Oracle
    import pandas as pd
    
    db = cx_Oracle.connect(db_id, db_pwd, db_ip) 
        
    sql = 'select * from ' + table_nm

    df = pd.read_sql(sql=sql, con = db) 
    df.columns = columns
    db.close
    
    return df


In [8]:
import re
import os

file_list = os.listdir(origin_path)
for file in file_list:
    kr_tbl_nm = re.findall('(\d\d).*\((.*)\)',file)
    print(kr_tbl_nm[0][0], kr_tbl_nm[0][1])

00 상권-영역
01 상권-추정유동인구
02 상권-집객시설
03 상권배후지-추정유동인구
04 상권배후지-집객시설
05 상권배후지-상주인구
06 상권_상주인구
07 상권배후지-아파트
08 상권배후지-점포
09 상권-상권변화지표
10 행정동별 상권변화지표
11 상권-추정매출
12 상권-직장인구
13 상권배후지-소득소비
14 상권배후지-직장인구
15 자치구별 상권변화지표
16 상권-아파트
18 상권-점포
19 상권배후지-추정매출


In [9]:
# import pandas as pd

# df = load_to_df('상권-추정유동인구')
# # df12 = load_to_df('상권-직장인구')
# df1 = df.copy()
# df1["상권/배후지"] = 0
# # print(df1.head())
# df1_1=df1[df1['기준_년_분기_코드']=='2019_1']
# df1_2=df1[df1['기준_년_분기_코드']=='2019_2']

In [39]:
import pandas as pd
def cat_df(sg, sg_hl): 
    df = load_to_df(sg)
    df0 = df.copy()
    df0["상권/배후지"] = 0
    
    df = load_to_df(sg_hl)
    df_hl = df.copy()
    df_hl["상권/배후지"] = 1
    
    new_columns = list(map( lambda s: s.replace('hl_',''), df_hl.columns ))
    df_hl.columns = new_columns
    
    return pd.concat([df0, df_hl])

In [40]:
df_1_3=cat_df('상권-추정유동인구','상권배후지-추정유동인구')

In [41]:
week = {'여성' : ['월요일','화요일','수요일','목요일','금요일'],
       '남성' : ['월요일','화요일','수요일','목요일','금요일'],}

weeks = {'여성' : ['토요일','일요일'], '남성':['토요일','일요일'],}

weekend = {'여성' : ['일요일','월요일','화요일','수요일','목요일','금요일','토요일'],
         '남성' : ['일요일','월요일','화요일','수요일','목요일','금요일','토요일'],}
delete_word = '시간대'

In [42]:
def delete_age_by_time(any_df, any_dict, any_word):
    for key, values in any_dict.items():
        for value in values:
            any_df[key+'_'+value+'_시간_유동인구수'] = any_df.filter(like=key).filter(like=value).sum(axis=1)
            del_columns = any_df.filter(like=key).filter(like=value).filter(like=any_word).columns
            any_df.drop(del_columns, axis=1, inplace=True)
                    
    return any_df

def delete_day1(any_df, any_dict):
    for key, values in any_dict.items():
        for value in values:
            any_df[key+'_주중_시간_유동인구수'] = any_df.filter(like=key).filter(like=value).sum(axis=1)
            del_columns = any_df.filter(like=key).filter(like=value).columns
            any_df.drop(del_columns, axis=1, inplace=True)            
    return any_df

def delete_day2(any_df, any_dict):
    for key, values in any_dict.items():
        for value in values:
            any_df[key+'_주말_시간_유동인구수'] = any_df.filter(like=key).filter(like=value).sum(axis=1)
            del_columns = any_df.filter(like=key).filter(like=value).columns
            any_df.drop(del_columns, axis=1, inplace=True)            
    return any_df

In [43]:
pop_time_sex_total = delete_age_by_time(df_1_3, weekend, delete_word)    # 여성, 남성 요일별, 시간대별 컬럼을 요일 인구로 합

In [46]:
# pop_time_sex_total.info()

In [47]:
pop_time_sex_total=pop_time_sex_total.drop(pop_time_sex_total.iloc[:,5:17], axis=1)

In [17]:
# pop_time_sex_total=pop_time_sex_total.set_index('상권_코드')#.filter(like='요일')

In [18]:
# pop_time_sex_total.info()

In [48]:
pop_s_total=pop_time_sex_total.copy()

In [49]:
pop_total=delete_day1(pop_s_total, week )

In [21]:
# pop_total.head()

In [50]:
pop_total = delete_day2(pop_total, weeks)

In [51]:
# pop_total.info()

In [52]:
# 비율 구하기
def ratio(df_col1,df_col2):
    total = df_col1+df_col2
    total2 = df_col1/total*100
    return total2

In [54]:
pop_total.head()

,기준_년_분기_코드,상권_코드,총_유동인구_수,남성_유동인구_수,여성_유동인구_수,월요일_유동인구_수,화요일_유동인구_수,수요일_유동인구_수,목요일_유동인구_수,금요일_유동인구_수,토요일_유동인구_수,일요일_유동인구_수,상권/배후지,여성_주중_시간_유동인구수,남성_주중_시간_유동인구수,여성_주말_시간_유동인구수,남성_주말_시간_유동인구수
0,2019_1,1000001,118266,61268,56997,15602,16948,17053,18090,19122,17650,13801,0,9103,10008,7228,6575
1,2019_1,1000002,246619,147358,99261,29952,29688,30735,31196,34814,42919,47315,0,14616,20191,16578,30739
2,2019_1,1000003,144480,80073,64407,19806,19937,21081,22238,24603,21131,15684,0,10823,13782,7294,8392
3,2019_1,1000012,166390,70339,96051,23431,23792,24378,23819,25957,23319,21699,0,15119,10842,11764,9943
4,2019_1,1000013,65518,33437,32080,7998,8808,9081,9785,11075,11064,7705,0,5415,5668,3836,3871


In [55]:
pop_total['여성_주중시간_유동인구_비율']=ratio(pop_total['여성_주중_시간_유동인구수'],pop_total['여성_주말_시간_유동인구수'])
pop_total['여성_주말시간_유동인구_비율']=ratio(pop_total['여성_주말_시간_유동인구수'],pop_total['여성_주중_시간_유동인구수'])

pop_total['남성_주중시간_유동인구_비율']=ratio(pop_total['남성_주중_시간_유동인구수'],pop_total['남성_주말_시간_유동인구수'])
pop_total['남성_주말시간_유동인구_비율']=ratio(pop_total['남성_주말_시간_유동인구수'],pop_total['남성_주중_시간_유동인구수'])

In [59]:
pop_total.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5010 entries, 0 to 2019
Data columns (total 21 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   기준_년_분기_코드       5010 non-null   object 
 1   상권_코드            5010 non-null   object 
 2   총_유동인구_수         5010 non-null   int64  
 3   남성_유동인구_수        5010 non-null   int64  
 4   여성_유동인구_수        5010 non-null   int64  
 5   월요일_유동인구_수       5010 non-null   int64  
 6   화요일_유동인구_수       5010 non-null   int64  
 7   수요일_유동인구_수       5010 non-null   int64  
 8   목요일_유동인구_수       5010 non-null   int64  
 9   금요일_유동인구_수       5010 non-null   int64  
 10  토요일_유동인구_수       5010 non-null   int64  
 11  일요일_유동인구_수       5010 non-null   int64  
 12  상권/배후지           5010 non-null   int64  
 13  여성_주중_시간_유동인구수   5010 non-null   int64  
 14  남성_주중_시간_유동인구수   5010 non-null   int64  
 15  여성_주말_시간_유동인구수   5010 non-null   int64  
 16  남성_주말_시간_유동인구수   5010 non-null   int64  
 17  여성_주중시간_유동인구_비

In [57]:
pop_total[pop_total['여성_주중시간_유동인구_비율'].isnull()]   #1001344
pop_total[pop_total['남성_주말시간_유동인구_비율'].isnull()]   #1000026
pop_total=pop_total.fillna(0)  # 비율에서 nan값 발생. 확인해 보니 0을 나눠서 그렇게됨. 0으로 값 대체함.

In [58]:
# pop_total.info()

In [60]:
pop_total['총_주중_유동인구수']=pop_total.iloc[:,5:10].sum(axis=1)
pop_total['총_주말_유동인구수']=pop_total.iloc[:,10:12].sum(axis=1)

In [61]:
pop_total=pop_total.drop(pop_total.iloc[:,5:12], axis=1)

In [62]:
pop_total.head()

,기준_년_분기_코드,상권_코드,총_유동인구_수,남성_유동인구_수,여성_유동인구_수,상권/배후지,여성_주중_시간_유동인구수,남성_주중_시간_유동인구수,여성_주말_시간_유동인구수,남성_주말_시간_유동인구수,여성_주중시간_유동인구_비율,여성_주말시간_유동인구_비율,남성_주중시간_유동인구_비율,남성_주말시간_유동인구_비율,총_주중_유동인구수,총_주말_유동인구수
0,2019_1,1000001,118266,61268,56997,0,9103,10008,7228,6575,55.740616,44.259384,60.350962,39.649038,86815,31451
1,2019_1,1000002,246619,147358,99261,0,14616,20191,16578,30739,46.855164,53.144836,39.644610,60.355390,156385,90234
2,2019_1,1000003,144480,80073,64407,0,10823,13782,7294,8392,59.739471,40.260529,62.153874,37.846126,107665,36815
3,2019_1,1000012,166390,70339,96051,0,15119,10842,11764,9943,56.240003,43.759997,52.162617,47.837383,121377,45018
4,2019_1,1000013,65518,33437,32080,0,5415,5668,3836,3871,58.534213,41.465787,59.419226,40.580774,46747,18769


In [63]:
pop_total['주중_유동인구_비율']=ratio(pop_total['총_주중_유동인구수'],pop_total['총_주말_유동인구수'])
pop_total['주말_유동인구_비율']=ratio(pop_total['총_주말_유동인구수'],pop_total['총_주중_유동인구수'])
pop_total['남성_유동인구_비율']=ratio(pop_total['남성_유동인구_수'],pop_total['여성_유동인구_수'])
pop_total['여성_유동인구_비율']=ratio(pop_total['여성_유동인구_수'],pop_total['남성_유동인구_수'])

In [69]:
pop_total.head()

,기준_년_분기_코드,상권_코드,총_유동인구_수,남성_유동인구_수,여성_유동인구_수,상권/배후지,여성_주중_시간_유동인구수,남성_주중_시간_유동인구수,여성_주말_시간_유동인구수,남성_주말_시간_유동인구수,여성_주중시간_유동인구_비율,여성_주말시간_유동인구_비율,남성_주중시간_유동인구_비율,남성_주말시간_유동인구_비율,총_주중_유동인구수,총_주말_유동인구수,주중_유동인구_비율,주말_유동인구_비율,남성_유동인구_비율,여성_유동인구_비율
0,2019_1,1000001,118266,61268,56997,0,9103,10008,7228,6575,55.740616,44.259384,60.350962,39.649038,86815,31451,73.406558,26.593442,51.805691,48.194309
1,2019_1,1000002,246619,147358,99261,0,14616,20191,16578,30739,46.855164,53.144836,39.644610,60.355390,156385,90234,63.411578,36.588422,59.751276,40.248724
2,2019_1,1000003,144480,80073,64407,0,10823,13782,7294,8392,59.739471,40.260529,62.153874,37.846126,107665,36815,74.518965,25.481035,55.421512,44.578488
3,2019_1,1000012,166390,70339,96051,0,15119,10842,11764,9943,56.240003,43.759997,52.162617,47.837383,121377,45018,72.945101,27.054899,42.273574,57.726426
4,2019_1,1000013,65518,33437,32080,0,5415,5668,3836,3871,58.534213,41.465787,59.419226,40.580774,46747,18769,71.352036,28.647964,51.035609,48.964391


In [70]:
pop_total[pop_total['상권_코드']=='1000001']

,기준_년_분기_코드,상권_코드,총_유동인구_수,남성_유동인구_수,여성_유동인구_수,상권/배후지,여성_주중_시간_유동인구수,남성_주중_시간_유동인구수,여성_주말_시간_유동인구수,남성_주말_시간_유동인구수,여성_주중시간_유동인구_비율,여성_주말시간_유동인구_비율,남성_주중시간_유동인구_비율,남성_주말시간_유동인구_비율,총_주중_유동인구수,총_주말_유동인구수,주중_유동인구_비율,주말_유동인구_비율,남성_유동인구_비율,여성_유동인구_비율
0,2019_1,1000001,118266,61268,56997,0,9103,10008,7228,6575,55.740616,44.259384,60.350962,39.649038,86815,31451,73.406558,26.593442,51.805691,48.194309
1221,2019_2,1000001,167204,82096,85105,0,12136,12478,11685,9226,50.946644,49.053356,57.491707,42.508293,118416,48780,70.824661,29.175339,49.100185,50.899815
45,2019_1,1000001,471903,253713,218188,1,35595,43907,26036,22734,57.755026,42.244974,65.885866,34.114134,359301,112607,76.137934,23.862066,53.764031,46.235969
796,2019_2,1000001,638841,326643,312201,1,46030,50977,39920,31716,53.554392,46.445608,61.646089,38.353911,470491,168358,73.646668,26.353332,51.130323,48.869677
